In [1]:
%autosave 60
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosaving every 60 seconds


# A Backtracking CSP-Solver

The module <tt>extractVariables</tt> implements the function $\texttt{extractVars}(e)$ that takes a *Python* expression $e$ as its argument and that returns the set if all variables occurring in $e$.

In [2]:
import extractVariables as ev

In [3]:
ev.extractVars('1.0 * x + y*z**2')

{'x', 'y', 'z'}

In [4]:
def arb(M):
    "Return some element from the set M."
    for x in M:
        return x

Backtracking is simulated by raising the <tt>Backtrack</tt> exception.  We define this new class of exception so that we can distinguish <tt>Backtrack</tt> exceptions from ordinary exceptions.

In [5]:
class Backtrack(Exception):
    pass

The function $\texttt{solve}(\texttt{CSP})$ tries to compute a solution for the *constraint satisfaction problem* $\texttt{CSP}$.  A *constraint satisfaction problem* is defined as a triple of the form
$$ \langle\texttt{Variables}, \texttt{Values}, \texttt{Constraints}\rangle $$
where
<ol>
    <li>$\texttt{Variables}$ is a list of the variables occurring in the CSP,</li>
    <li>$\texttt{Values}$ is the set of values that can be substitued for the variables, and</li>
    <li>$\texttt{Constraints}$ is the set of formulas that need to be satisfied.</li>
</ol>

In [6]:
def solve(CSP):
    "Compute a solution for the given constraint satisfaction problem."
    Variables, Values, Constraints = CSP
    CSP = (Variables, Values, [(f, ev.extractVars(f) & set(Variables)) for f in Constraints])
    try:
        return backtrack_search({}, CSP)
    except Backtrack:
        return  # no solution found

In [7]:
def backtrack_search(Assignment, CSP):
    """
    Given a partial variable assignment, this function tries to complete this assignment
    towards a solution of the CSP.
    """
    (Variables, Values, Constraints) = CSP
    if len(Assignment) == len(Variables):
        return Assignment
    var = [x for x in Variables if x not in Assignment][0]
    for value in Values:
        try:
            if isConsistent(var, value, Assignment, Constraints):
                NewAssign      = Assignment.copy()
                NewAssign[var] = value
                return backtrack_search(NewAssign, CSP)
        except Backtrack:
            continue
    # all values have been tried without success, no solution has been found
    raise Backtrack()

The function $\texttt{isConsistent}(\texttt{var}, \texttt{value}, \texttt{Assignment}, \texttt{CSP})$ takes for arguments:
<ol>
    <li>$\texttt{var}$ is a variable that does not occur in $\texttt{Assignment}$,</li>
    <li>$\texttt{value}$ is a value that can be substituted for this variable,</li>
    <li>$\texttt{Assignment}$ is a partial variable assignment, and </li>
    <li>$\texttt{CSP}$ is a constraint satisfaction problem.</li>
</ol>
This function returns <tt>True</tt> iff the partial variable assignment 
$$\texttt{Assignment} \cup \{\langle\texttt{var} \mapsto\texttt{value}\rangle\}$$
is consistent with all the constraints occurring in $\texttt{CSP}$.

In [8]:
def isConsistent(var, value, Assignment, Constraints):
    NewAssign      = Assignment.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if var in Vs and Vs <= NewAssign.keys()
              )

The function $\texttt{queensCSP}$ returns a CSP that codes the 8-Queens-Problem.

## The 8-Queens-Problem

The function $\texttt{queensCSP}$ returns a CSP that codes the 8-Queens-Problem.  For $i\in\{1,\cdots,8\}$, the variable $\texttt{Q}i$ specifies the column of the queen that is placed in row $i$.

In [9]:
def queensCSP():
    "Returns a CSP coding the 8 queens problem."
    S            = range(1, 8+1)          # used as indices
    Variables    = [ f'Q{i}' for i in S ]
    Values       = { 1, 2, 3, 4, 5, 6, 7, 8 }
    SameRow      = { f'Q{i} != Q{j}' for i in S for j in S if i < j }
    SameDiagonal = { f'abs(Q{i}-Q{j})!={j-i}' for i in S for j in S if i < j }
    return (Variables, Values, SameRow | SameDiagonal)

In [10]:
queensProblem = queensCSP()

In [11]:
queensProblem

(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8'],
 {1, 2, 3, 4, 5, 6, 7, 8},
 {'Q1 != Q2',
  'Q1 != Q3',
  'Q1 != Q4',
  'Q1 != Q5',
  'Q1 != Q6',
  'Q1 != Q7',
  'Q1 != Q8',
  'Q2 != Q3',
  'Q2 != Q4',
  'Q2 != Q5',
  'Q2 != Q6',
  'Q2 != Q7',
  'Q2 != Q8',
  'Q3 != Q4',
  'Q3 != Q5',
  'Q3 != Q6',
  'Q3 != Q7',
  'Q3 != Q8',
  'Q4 != Q5',
  'Q4 != Q6',
  'Q4 != Q7',
  'Q4 != Q8',
  'Q5 != Q6',
  'Q5 != Q7',
  'Q5 != Q8',
  'Q6 != Q7',
  'Q6 != Q8',
  'Q7 != Q8',
  'abs(Q1-Q2)!=1',
  'abs(Q1-Q3)!=2',
  'abs(Q1-Q4)!=3',
  'abs(Q1-Q5)!=4',
  'abs(Q1-Q6)!=5',
  'abs(Q1-Q7)!=6',
  'abs(Q1-Q8)!=7',
  'abs(Q2-Q3)!=1',
  'abs(Q2-Q4)!=2',
  'abs(Q2-Q5)!=3',
  'abs(Q2-Q6)!=4',
  'abs(Q2-Q7)!=5',
  'abs(Q2-Q8)!=6',
  'abs(Q3-Q4)!=1',
  'abs(Q3-Q5)!=2',
  'abs(Q3-Q6)!=3',
  'abs(Q3-Q7)!=4',
  'abs(Q3-Q8)!=5',
  'abs(Q4-Q5)!=1',
  'abs(Q4-Q6)!=2',
  'abs(Q4-Q7)!=3',
  'abs(Q4-Q8)!=4',
  'abs(Q5-Q6)!=1',
  'abs(Q5-Q7)!=2',
  'abs(Q5-Q8)!=3',
  'abs(Q6-Q7)!=1',
  'abs(Q6-Q8)!=2',
  'abs(Q7-Q8)!=1'

In [12]:
import time

In [13]:
start    = time.time()
Solution = solve(queensProblem)
stop     = time.time()
print(f'Time needed: {round((stop-start) * 1000)} milliseconds.')

Time needed: 48 milliseconds.


In [14]:
print(Solution)

{'Q1': 1, 'Q2': 5, 'Q3': 8, 'Q4': 6, 'Q5': 3, 'Q6': 7, 'Q7': 2, 'Q8': 4}


In [15]:
def printSolution(Assignment):
    if Assignment == None:
        print('no solution found')
        return
    n      = len(Assignment)
    Queens = [0] * (n+1)
    for row in range(1, n+1):
        Queens[row] = Assignment[f'Q{row}']
    print((4 * n + 1) * "-")
    for row in range(1, n+1):
        line = "|"
        for col in range(1, n+1):
            if Queens[row] == col:
                line += " Q |"
            else:
                line += "   |"
        print(line)
        print((4*n+1) * "-")

In [16]:
printSolution(Solution)

---------------------------------
| Q |   |   |   |   |   |   |   |
---------------------------------
|   |   |   |   | Q |   |   |   |
---------------------------------
|   |   |   |   |   |   |   | Q |
---------------------------------
|   |   |   |   |   | Q |   |   |
---------------------------------
|   |   | Q |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   | Q |   |
---------------------------------
|   | Q |   |   |   |   |   |   |
---------------------------------
|   |   |   | Q |   |   |   |   |
---------------------------------
